### _Setup_

In [ ]:
# Reset memory
%reset -f

In [ ]:
# Packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import List, Union, Tuple
from sklearn.cross_decomposition import PLSRegression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, r2_score
import time
from kneed import KneeLocator
from sklearn.model_selection import StratifiedKFold

In [ ]:
# Load Data
df = pd.read_csv('data.csv')

### _Functions_

In [ ]:
def find_col_index_of_spectra(
    df: pd.DataFrame
) -> int:
    """
    Find the column index where spectral data starts.

    Assumes spectral column names can be converted to float (e.g., "730.5", "731.0").

    Parameters:
        df : Input DataFrame

    Returns:
        Index of the first spectral column, or -1 if not found.
    """
    for idx, col in enumerate(df.columns):
        try:
            float(col)
            return idx
        except (ValueError, TypeError):
            continue
    return -1

def split_train_test(
    df: pd.DataFrame,
    test_variety: str,
    test_season: int       
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    """
    Split a DataFrame into one training set and two test sets:

    - Variety test set: Variety == test_variety AND Year == 2024
    - Season test set : Year == test_season 

    The training set excludes all rows that belong to any of the test sets.
    The season test set only includes varieties that are present in the training set.

    Parameters:
        df           : Full pandas DataFrame
        test_variety : Variety used for the test set
        test_season  : Year used for the season test

    Returns:
        df_train        : Training set
        df_test_variety : Test set for specified variety and 2024
        df_test_season  : Test set for specified season (filtered by train varieties)
    """

    # Select test set for the specified variety in year 2024
    df_test_variety = df[
        (df["Variety"] == test_variety) &
        (df["Scan Date Year"] == 2024)
    ]

    # Select test set for the specified season (regardless of variety)
    df_test_season = df[
        df["Scan Date Year"] == test_season
    ]

    # Select training set (exclude test variety and test season)
    df_train = df[
        (df["Variety"] != test_variety) &
        (df["Scan Date Year"] != test_season)
    ]

    # Filter season test set to only include varieties present in training set
    train_varieties = df_train["Variety"].unique()
    df_test_season = df_test_season[
        df_test_season["Variety"].isin(train_varieties)
    ]

    return df_train, df_test_variety, df_test_season

def split_x_y(
    df: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Split train and test DataFrames into x (spectra features) and y (target) arrays.
    Assumes find_col_index_of_spectra() is defined globally.

    Parameters:
        df_train: Training set DataFrame.
        df_test : Test set DataFrame.

    Returns:
        x_train: NumPy array of training features.
        y_train: NumPy array of training targets.
    """
    spectra_cols = list(df.columns[find_col_index_of_spectra(df):])
    target_cols = ['Brix (Position)']

    x = df[spectra_cols].values
    y = df[target_cols].values

    return (
        x,
        y
    )

def stratified_cv_splits(
    x_train, 
    y_train, 
    n_splits, 
    random_state, 
    n_bins=10
):
    """
    Generate stratified K-Fold splits for regression by binning y_train into quantiles.

    Parameters:
        x_train      : Feature matrix (NumPy array or DataFrame)
        y_train      : Target values (NumPy array or Series)
        n_splits     : Number of folds for StratifiedKFold
        random_state : Random seed for reproducibility
        n_bins       : Number of quantile bins to stratify target into (default: 10)

    Returns:
        Generator of (train_idx, val_idx) tuples
    """

    # Bin the target into quantiles
    y_binned = pd.qcut(np.ravel(y_train), q=n_bins, labels=False, duplicates='drop')

    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)

    for train_idx, val_idx in skf.split(x_train, y_binned):
        yield train_idx, val_idx

def model_irpls(
    x_data: np.ndarray,
    y_data: np.ndarray,
    n_components: int,
    max_iter: int,
    tol: float,
    c: float,
    verbose: bool = False
) -> tuple[PLSRegression, np.ndarray]:
    """
    Iteratively Reweighted PLS with per‐component deflation (irPLS).

    This follows the MATLAB irpls.m structure: for each latent component,
    we iteratively update a diagonal weight matrix D until its sum converges,
    then deflate X and Y, and move on to the next component.

    Parameters:
        x_data       : (n_samples, n_features) predictor matrix
        y_data       : (n_samples,) or (n_samples,1) response vector
        n_components : number of PLS latent components to extract
        max_iter     : max reweighting iterations per component
        tol          : tolerance on change in sum(weights) for convergence
        c            : Tukey bisquare tuning constant
        verbose      : if True, print per‐component convergence info

    Returns:
        final_pls   : PLSRegression fitted on full weighted X,Y
        final_wts   : final sample weights from last component (n_samples,)
    """
    # Flatten y and median‐center X and Y for robustness
    y = y_data.flatten()
    x = x_data.copy()
    x = x - np.median(x, axis=0)
    y = y - np.median(y)

    n, p = x.shape

    # Initialize diagonal weight matrix D as identity scaled by 1/n
    D = np.eye(n) * (1.0 / n)

    # Prepare matrices to store W, T, P, Q across components
    W = np.zeros((p, n_components))     # weight vectors
    T = np.zeros((n, n_components))     # score vectors
    P = np.zeros((p, n_components))     # loadings
    Q = np.zeros(n_components)          # Y loadings

    for comp in range(n_components):
        if verbose:
            print(f"\n--- Component {comp+1}/{n_components} ---")
        
        # Initialize uniform weights for first iteration
        d = np.full(n, 1.0/n)

        for iteration in range(1, max_iter+1):
            # Compute weighted covariance vector: v_t = X' D y
            vt = x.T.dot(d * y)

            # Normalize to get component weight vector
            w = vt / np.linalg.norm(vt)

            # Score vector (project data onto w)
            t = x.dot(w)
            t = t / np.linalg.norm(t)

            # Regression coefficient on t
            q = float((d * y).dot(t))

            # Compute residual for robust weight update
            r = y - t * q

            # Estimate robust scale via MAD
            med = np.median(r)
            mad = np.median(np.abs(r - med))
            scale = (mad * 1.4826) if mad != 0 else np.std(r)

            # Apply Tukey bisquare weight function
            u = r / scale
            new_d = np.where(np.abs(u) < c, (1 - (u**2)/(c**2))**2, 0.0)

            # Check for convergence of weight vector
            if np.abs(new_d.sum() - d.sum()) < tol:
                if verbose:
                    print(f" converged inner at iter {iteration}, Δsum={np.abs(new_d.sum()-d.sum()):.3e}")
                d = new_d
                break

            d = new_d
        else:
            if verbose:
                print(f" inner did not converge after {max_iter} its; last Δsum={np.abs(new_d.sum()-d.sum()):.3e}")

        # Store learned component vectors
        W[:, comp] = w
        T[:, comp] = t
        Q[comp]    = q
        P[:, comp] = x.T.dot(d * t)

        # Deflate X and Y for next component
        x = x - np.outer(t, P[:, comp])
        y = y - t * q

    # Compute final PLS model using sample weights from last component
    final_weights = d
    sqrt_w = np.sqrt(final_weights)
    Xw = x_data * sqrt_w[:, None]     # weighted X
    Yw = y_data.flatten() * sqrt_w    # weighted Y

    # Fit final PLS regression model
    final_pls = PLSRegression(n_components=n_components)
    final_pls.fit(Xw, Yw)

    return final_pls, final_weights

def train_cross_validation_gridsearch(
    x_train_data: np.ndarray,
    y_train_data: np.ndarray,
    c_values: List[float],
    max_components: int,
    n_splits: int,
    random_state: int,
    verbose: bool = False,
    n_bins: int = 10
) -> Tuple[pd.DataFrame, float, int, float]:
    """
    Grid search over c values using stratified K-fold CV for regression.

    Parameters:
        x_train_data   : Training features
        y_train_data   : Training targets
        c_values       : List of c values to test
        max_components : Max number of latent variables to test
        n_splits       : Number of folds for StratifiedKFold
        random_state   : Random seed
        verbose        : Print progress
        n_bins         : Number of bins to stratify y (default = 10)

    Returns:
        cv_results_df  : DataFrame with [c, knee_LV, knee_RMSECV]
        cv_opt_rmsecv  : Best RMSECV found
        cv_opt_A       : Optimal number of LVs
        cv_opt_c       : Optimal c value
    """
    results = []

    # Loop over all candidate c values
    for c in c_values:
        if verbose:
            print(f"\nTesting c = {c:.2f}")

        rmsecv_list = []

        # Loop over number of latent variables to test
        for n_comp in range(1, max_components + 1):
            mse_folds = []

            # K-fold stratified CV
            for train_idx, val_idx in stratified_cv_splits(x_train_data, y_train_data, n_splits, random_state, n_bins):
                X_train_fold = x_train_data[train_idx]
                X_val_fold   = x_train_data[val_idx]
                y_train_fold = y_train_data[train_idx]
                y_val_fold   = y_train_data[val_idx]

                # Train irPLS model for given c and component count
                pls_model, _ = model_irpls(
                    x_data=X_train_fold,
                    y_data=y_train_fold,
                    n_components=n_comp,
                    max_iter=100,
                    tol=1e-6,
                    c=c,
                    verbose=False
                )

                # Predict and compute fold MSE
                y_pred_val = pls_model.predict(X_val_fold).flatten()
                mse_folds.append(mean_squared_error(y_val_fold, y_pred_val))

            # Compute RMSECV for this component count
            rmsecv = np.sqrt(np.mean(mse_folds))
            rmsecv_list.append(rmsecv)

            if verbose:
                print(f"    LVs: {n_comp}, RMSECV: {rmsecv:.4f}")

        # Plot RMSECV curve for this c value
        plt.figure(figsize=(8, 5))
        plt.plot(range(1, max_components + 1), rmsecv_list, marker='o')
        plt.xlabel("Number of Latent Variables")
        plt.ylabel("RMSECV")
        plt.title(f"RMSECV Curve for c = {c:.2f}")
        plt.grid(True)
        plt.show()

        # Find the knee point (optimal number of components)
        x_vals = list(range(1, max_components + 1))
        knee_locator = KneeLocator(x_vals, rmsecv_list, curve="convex", direction="decreasing")
        knee_LV = knee_locator.knee

        # Fallback to min RMSECV if no knee found
        if knee_LV is None:
            if verbose:
                print("No knee found, using min RMSECV as fallback.")
            knee_LV = np.argmin(rmsecv_list) + 1

        # Store result for this c value
        knee_rmsecv = rmsecv_list[knee_LV - 1]
        results.append({
            "c": c,
            "knee_LV": knee_LV,
            "knee_RMSECV": knee_rmsecv
        })

    # Compile results across all c values
    cv_results_df = pd.DataFrame(results)

    # Find best overall c value and corresponding model settings
    best_idx = cv_results_df["knee_RMSECV"].idxmin()
    cv_opt_c = cv_results_df.loc[best_idx, "c"]
    cv_opt_A = cv_results_df.loc[best_idx, "knee_LV"]
    cv_opt_rmsecv = cv_results_df.loc[best_idx, "knee_RMSECV"]

    if verbose:
        print("\n==== Best Overall ====")
        print(f"Best c: {cv_opt_c:.2f}")
        print(f"Best number of LVs: {cv_opt_A}")
        print(f"Best RMSECV: {cv_opt_rmsecv:.4f}")

    return cv_results_df, cv_opt_rmsecv, cv_opt_A, cv_opt_c

def test_irpls_model(
    x_train_data: np.ndarray,
    y_train_data: np.ndarray,
    x_test_data: np.ndarray,
    y_test_data: np.ndarray,
    opt_A: int,
    opt_c: float,
    max_iter: int,
    tol: float,
    verbose: bool = False
) -> Tuple[
    PLSRegression,  # irpls_model
    pd.DataFrame,   # test_irpls_results
    float,          # test_rmsep
    float,          # test_r2
    float,          # test_practical_accuracy
    float           # final_weights
]:
    """
    Train a final irPLS regression model on the training set using the specified number 
    of latent variables and bisquare tuning constant, then evaluate its performance on 
    the test set.
    """
    # Fit irPLS model on the full training set
    irpls_model, final_weights = model_irpls(
        x_data=x_train_data,
        y_data=y_train_data,
        n_components=opt_A,
        max_iter=max_iter,
        tol=tol,
        c=opt_c,
        verbose=verbose
    )

    # Predict on test set
    y_pred = irpls_model.predict(x_test_data).flatten()

    # Flatten true labels
    y_true = y_test_data.flatten()

    # Compute RMSEP (Root Mean Squared Error of Prediction)
    test_rmsep = float(np.sqrt(mean_squared_error(y_true, y_pred)))

    # Compute R² (coefficient of determination)
    test_r2 = float(r2_score(y_true, y_pred))

    # Compute practical accuracy: percentage of predictions within ±20% of ground truth
    pct_error = np.abs(y_pred - y_true) / np.abs(y_true)
    test_practical_accuracy = float((pct_error <= 0.2).mean() * 100.0)

    # Assemble result DataFrame with observed, predicted, and percentage error
    test_irpls_results = pd.DataFrame({
        "observed":  y_true,
        "predicted": y_pred,
        "pct_error": pct_error
    })

    # Print test performance summary
    print(f"Test RMSEP: {test_rmsep:.4f}")
    print(f"Test R²: {test_r2:.4f}")
    print(f"Practical accuracy (±20%): {test_practical_accuracy:.1f}%")

    # Plot parity plot (Observed vs Predicted)
    plt.figure(figsize=(8, 6))
    plt.scatter(y_true, y_pred, alpha=0.7, label="Test Data")
    plt.plot(
        [y_true.min(), y_true.max()],
        [y_true.min(), y_true.max()],
        "k--", lw=2, label="Ideal"
    )
    plt.xlabel("Observed")
    plt.ylabel("Predicted")
    plt.title("Observed vs. Predicted on Test Set (irPLS)")
    plt.legend()
    plt.grid(True)
    plt.show()

    return (
        irpls_model,
        test_irpls_results,
        test_rmsep,
        test_r2,
        test_practical_accuracy,
        final_weights
    )

### _Variables_

In [ ]:
DF = df
RANDOM_STATE = 27
TEST_VARIETY = "TestVariety"
TEST_SEASON = 2025
MAX_COMPONENTS = 20
N_SPLITS = 3
MAX_ITER = 100
C_RANGE = list(range(1, 31))
TOL = 1e-6

### _Run_

In [ ]:
# === Split into train and test sets ===
df_train, df_test_variety, df_test_season = split_train_test(
    df,
    test_variety=TEST_VARIETY,
    test_season=TEST_SEASON
)

# === Convert to x and y arrays ===
x_train, y_train = split_x_y(
    df_train,
)
x_test_variety, y_train_variety = split_x_y(
    df_test_variety,
)
x_test_season, y_train_season = split_x_y(
    df_test_season,
)

# === Run gridsearch cross-validation ===
cv_results_df, cv_opt_rmsecv, cv_opt_A, cv_opt_c = train_cross_validation_gridsearch(
    x_train_data=x_train,
    y_train_data=y_train,
    c_values=C_RANGE,
    max_components=MAX_COMPONENTS,
    n_splits=N_SPLITS,
    random_state=RANDOM_STATE,
    verbose=True
)

print(f"Optimal # of components: {cv_opt_A} | Optimal c: {cv_opt_c:.2f} | CV RMSECV: {cv_opt_rmsecv:.4f}")

# === Test on VARIETY ===
irpls_model_variety, results_variety, rmsep_variety, r2_variety, acc_variety, weights_variety = test_irpls_model(
    x_train_data=x_train,
    y_train_data=y_train,
    x_test_data=x_test_variety,
    y_test_data=y_train_variety,
    opt_A=cv_opt_A,
    opt_c=cv_opt_c,
    max_iter=MAX_ITER,
    tol=TOL,
    verbose=True
)

# === Test on SEASON ===
irpls_model_season, results_season, rmsep_season, r2_season, acc_season, weights_season = test_irpls_model(
    x_train_data=x_train,
    y_train_data=y_train,
    x_test_data=x_test_season,
    y_test_data=y_train_season,
    opt_A=cv_opt_A,
    opt_c=cv_opt_c,
    max_iter=MAX_ITER,
    tol=TOL,
    verbose=True
)

# === Print summary ===
print("\n==== irPLS Test Results ====")
print(f"Variety  | RMSEP: {rmsep_variety:.4f} | R²: {r2_variety:.4f} | Acc: {acc_variety:.2%}")
print(f"Season   | RMSEP: {rmsep_season:.4f}  | R²: {r2_season:.4f}  | Acc: {acc_season:.2%}")

### _Inference Time Analysis_

In [ ]:
def get_inference_sample_set(
    df_variety: pd.DataFrame,
    df_season: pd.DataFrame,
    random_state: int,
    sample_size: int = 1000
) -> Tuple[np.ndarray, np.ndarray]:
    """
    Combine two test sets (variety and season), sample rows randomly, and return X and y arrays.

    Parameters:
        df_variety   : DataFrame for variety-based test set
        df_season    : DataFrame for season-based test set
        random_state : Random seed for reproducibility
        sample_size  : Number of rows to sample from combined test set

    Returns:
        x_sample : NumPy array of shape (sample_size, n_features) with spectral features
        y_sample : NumPy array of shape (sample_size,) with corresponding Brix values
    """
    # Combine the two test sets
    df_combined = pd.concat([df_variety, df_season], axis=0)

    # Randomly sample rows from the combined test set
    df_sample = df_combined.sample(
        n=sample_size,
        random_state=random_state
    )

    # Split into X and y arrays
    x_sample, y_sample = split_x_y(df_sample)

    return x_sample, y_sample

def test_irpls_inference_time(
    x_train: np.ndarray,
    y_train: np.ndarray,
    x_test: np.ndarray,
    y_test: np.ndarray,
    n_components: int,
    c: float,
    max_iter: int,
    tol: float
) -> float:
    """
    Train irPLS with specified hyperparameters and return average inference time per sample (in ms).

    Parameters:
        x_train      : Training features
        y_train      : Training targets
        x_test       : Test features
        y_test       : Test targets
        n_components : Number of irPLS latent variables
        c            : Tukey bisquare tuning constant
        max_iter     : Maximum number of iterations per component
        tol          : Convergence threshold for weight updates

    Returns:
        avg_inference_time_ms : Average inference time per sample in milliseconds
    """
    # Fit irPLS model on the training data
    irpls_model, _ = model_irpls(
        x_data=x_train,
        y_data=y_train,
        n_components=n_components,
        max_iter=max_iter,
        tol=tol,
        c=c,
        verbose=False
    )

    times = []

    # Predict each test sample individually and measure the time
    for x in x_test:
        x_input = x.reshape(1, -1)
        start = time.time()
        _ = irpls_model.predict(x_input)
        end = time.time()
        times.append(end - start)

    # Compute average inference time in milliseconds
    avg_inference_time_ms = np.mean(times) * 1000

    # Print result
    print(f"Average inference time (irPLS): {avg_inference_time_ms:.3f} ms/sample")

    return avg_inference_time_ms


In [ ]:
# === Create sample set for inference time measurement ===
x_inference_time, y_inference_time = get_inference_sample_set(
    df_test_variety,
    df_test_season,
    random_state=RANDOM_STATE
)

# === Compute the average inference time ===
inference_time_irpls = test_irpls_inference_time(
    x_train,
    y_train,
    x_inference_time,
    y_inference_time,
    n_components=cv_opt_A,
    c=cv_opt_c,
    max_iter=MAX_ITER,
    tol=TOL
)
